In [ ]:
#Splitting the Dataframe into X_features and the Y_Classes
"""
WINDOW_SIZE = 5
[
  [[dp1, dh1, dl1], [ds2, dp2], [ds3, dp3], [ds4, dp4], [ds5, dp5]],
  [[ds2, dp2], [ds3, dp3], [ds4, dp4], [ds5, dp5], [ds6, dp6]]  
]
"""

def __generate_x_y_from_nsc_df(__data_frame , time_step=5):
    data_array = __data_frame.to_numpy()
    X = []
    y = []
    for i in range(len(data_array)-time_step):
        
        rows = data_array[i : i + time_step]  #Select 5 rows in each and every iteration
        zipped_rows = [row for row in rows]  
        
        label = data_array[i + time_step][0] # Select the sixth row and the first column in that row . NB: Anything that is separated by [] makes rows and not columns .
        X.append(zipped_rows)
        y.append(label)
        
    return np.array(X).astype(np.float32), np.array(y).astype(np.float32).reshape(-1,1)

X_data, Y_classes = __generate_x_y_from_nsc_df(preprocessed_df , time_step=5)

print("X_Data shape : ",X_data.shape)
print("Y_Data shape : ",Y_classes.shape)

Y_classes



In [ ]:
#Model Creation and Model Compilation.
dropout_rate = 0.2

input_shape=(X_data.shape[1], X_data.shape[2])

print("Input shape : ",input_shape)


nsc_lstm_model = Sequential([
    Input(input_shape),
    LSTM(units=150), 
    Dense(units=1, activation='linear')
])

nsc_lstm_model.compile(loss=MeanSquaredError(), 
              optimizer=Adam(learning_rate=0.0001),
              metrics=[RootMeanSquaredError()])

plot_model(nsc_lstm_model, show_shapes=True, show_layer_names=True)


In [ ]:

#++++++++++++++++++++++++++++++++++++++++++   SPLITTING DATA +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#

def dynamic_data_split(X, y, ratio_train):
    """
    Splits data and labels into training, validation, and test sets.

    Args:
        X (a 3D numpy.ndarray): Data.
        y (numpy.ndarray): Labels.
        ratio_train (float): Ratio for training data.

    Returns:
        tuple: (X_train, y_train, X_val, y_val, X_test, y_test).
    """
    total_samples = X.shape[0]

    train_samples = int(ratio_train * total_samples)
    

    X_train, y_train = X[:train_samples], y[:train_samples]
    X_test, y_test = X[train_samples:], y[train_samples:]

    return X_train, y_train, X_test, y_test


train_ratio = 0.8
test_ratio = 0.2

X_train, y_train, X_test, y_test = dynamic_data_split(X_data,Y_classes , train_ratio)

X_train.shape,y_train.shape, X_test.shape, y_test.shape


In [ ]:

# __nsc_model_checkpoint = ModelCheckpoint('nsc_model/', save_best_only=True)


nsc_lstm_model.fit(X_train, y_train,batch_size=30, epochs=100, callbacks=[History()])

In [ ]:

test_predictions = nsc_lstm_model.predict(X_test).flatten()

test_df = pd.DataFrame(data={'Actuals':y_test.flatten(),'Test Predictions':test_predictions})

plt.plot(test_df['Actuals'], color = 'red', label = random_company_code + ' Stock Price')
plt.plot(test_df['Test Predictions'], color = 'green', label = 'Predicted ' + random_company_code+' Stock Price')
plt.title(random_company_code + ' Ltd. stock market predictions')
plt.xlabel('Time')
plt.ylabel( random_company_code + 'Stock Price')
plt.legend()
plt.show()

test_df
